In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from keras.layers import Lambda, Input, Dense
from keras.models import Model
from keras.datasets import mnist
from keras.losses import mse, binary_crossentropy
from keras.utils import plot_model
from keras import backend as K


import numpy as np 

import pandas as pd
import matplotlib.pyplot as plt
import argparse
import os

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from sklearn.metrics import log_loss, accuracy_score
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import Grid
%matplotlib inline

#import tensorflow as tf
#import numpy as np 
#import pandas as pd
#import matplotlib.pyplot as plt
#import os

#getting required files
r_df = pd.read_csv('/content/drive/My Drive/ml-1m.train.rating.csv', sep='\t', header=None, names=['user', 'item'], usecols=[0, 1], dtype={0: np.int32, 1: np.int32, 2:np.int32})
print(r_df.head())
r=np.array(r_df, dtype=int)[:,:]

r_t_df = pd.read_csv('/content/drive/My Drive/ml-1m.test.rating.csv',sep='\t',  header=None, names=['user', 'item'], usecols=[0, 1], dtype={0: np.int32, 1: np.int32, 2:np.int32})
print(r_t_df.head())
r_t=np.array(r_t_df, dtype=int)[:,:]

#checking if it can acces files
print(r.shape)

#getting max no of hotels and users
nb_users = int(max(max(r[:, 0]), max(r_t[:, 0])))
nb_movies = int(max(max(r[:, 1]), max(r_t[:, 1])))

# printing the number of users and movies
print(f'The number of users {nb_users}, The number of movies {nb_movies}')



   user  item
0     0    32
1     0    34
2     0     4
3     0    35
4     0    30
   user  item
0     0    25
1     1   133
2     2   207
3     3   208
4     4   222
[ 0 34]
The number of users 6039, The number of movies 3705


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
def convert_fn(x ):
    converted_data = []
    data=x
    for user_id in range(1, nb_users + 1):
        # getting all the movies ids that rated by every user
        movies_for_user_id = data[:, 1][data[:, 0] == user_id] # getting the movies ids that taken be the current user in the for loop
        rating_for_user_id = data[:, 2][data[:, 0] == user_id]
        ratings = np.zeros(nb_movies) # initialze all the ratings with zeros then include the rated movies
        ratings[movies_for_user_id - 1] = rating_for_user_id 
        converted_data.append(list(ratings))
    return converted_data

In [ ]:
train=convert_fn(r)
test=convert_fn(r_t)
y = np.asarray(train) 
y_t = np.asarray(test) 

In [ ]:
r_df .max(axis = 0)

user    6039
item    3705
dtype: int32

In [ ]:
print(y.shape,y_t.shape)

(6040, 3952) (6040, 3952)


In [ ]:
class RBM(object):
    
    def __init__(self, input_size, output_size, 
                 learning_rate, epochs, batchsize):
        # Define hyperparameters
        self._input_size = input_size
        self._output_size = output_size
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.batchsize = batchsize
        
        # Initialize weights and biases using zero matrices
        self.w = np.zeros([input_size, output_size], dtype=np.float32)
        self.hb = np.zeros([output_size], dtype=np.float32)
        self.vb = np.zeros([input_size], dtype=np.float32)
    # forward pass, where h is the hidden layer and v is the visible layer
    def prob_h_given_v(self, visible, w, hb):
        return tf.nn.sigmoid(tf.matmul(visible, w) + hb)
    # backward pass
    def prob_v_given_h(self, hidden, w, vb):
        return tf.nn.sigmoid(tf.matmul(hidden, tf.transpose(w)) + vb)
    # sampling function
    def sample_prob(self, probs):
        return tf.nn.relu(tf.sign(probs - tf.random_uniform(tf.shape(probs))))

    def train(self, X):
        _w = tf.placeholder(tf.float32, [self._input_size, self._output_size])
        _hb = tf.placeholder(tf.float32, [self._output_size])
        _vb = tf.placeholder(tf.float32, [self._input_size])
        
        prv_w = np.zeros([self._input_size, self._output_size], dtype=np.float32)
        prv_hb = np.zeros([self._output_size], dtype=np.float32)
        prv_vb = np.zeros([self._input_size], dtype=np.float32)
        
        cur_w = np.zeros([self._input_size, self._output_size], dtype=np.float32)
        cur_hb = np.zeros([self._output_size], dtype=np.float32)
        cur_vb = np.zeros([self._input_size], dtype=np.float32)
        
        v0 = tf.placeholder(tf.float32, [None, self._input_size])
        h0 = self.sample_prob(self.prob_h_given_v(v0, _w, _hb))
        v1 = self.sample_prob(self.prob_v_given_h(h0, _w, _vb))
        h1 = self.prob_h_given_v(v1, _w, _hb)
        #To update the weights, we perform constrastive divergence.
        positive_grad = tf.matmul(tf.transpose(v0), h0)
        negative_grad = tf.matmul(tf.transpose(v1), h1)
        
        update_w = _w + self.learning_rate * (positive_grad - negative_grad) / tf.to_float(tf.shape(v0)[0])
        update_vb = _vb +  self.learning_rate * tf.reduce_mean(v0 - v1, 0)
        update_hb = _hb +  self.learning_rate * tf.reduce_mean(h0 - h1, 0)
        # We also define the error as the MSE
        err = tf.reduce_mean(tf.square(v0 - v1))
        
        error_list = []
        '''Once we call sess.run, we can feed in batches of data to begin the training. 
           During the training, forward and backward passes will be made, and the RBM 
           will update weights based on how the generated data compares to the original input. 
           We will print the reconstruction error from each epoch'''
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            
            for epoch in range(self.epochs):
                for start, end in zip(range(0, len(X),  self.batchsize),range(self.batchsize,len(X), self.batchsize)):
                    batch = X[start:end]
                    cur_w = sess.run(update_w, feed_dict={v0: batch, _w: prv_w, _hb: prv_hb, _vb: prv_vb})
                    cur_hb = sess.run(update_hb, feed_dict={v0: batch,  _w: prv_w, _hb: prv_hb, _vb: prv_vb})
                    cur_vb = sess.run(update_vb, feed_dict={v0: batch, _w: prv_w, _hb: prv_hb, _vb: prv_vb})
                    prv_w = cur_w
                    prv_hb = cur_hb
                    prv_vb = cur_vb
                error = sess.run(err, feed_dict={v0: X, _w: cur_w, _vb: cur_vb, _hb: cur_hb})
                print ('Epoch: %d' % epoch,'reconstruction error: %f' % error)
                error_list.append(error)
            self.w = prv_w
            self.hb = prv_hb
            self.vb = prv_vb
            return error_list
    #function to generate new images from the generative model that the RBM has learned
    def rbm_output(self, X):
        
        input_X = tf.constant(X)
        _w = tf.constant(self.w)
        _hb = tf.constant(self.hb)
        _vb = tf.constant(self.vb)
        out = tf.nn.sigmoid(tf.matmul(input_X, _w) + _hb)
        hiddenGen = self.sample_prob(self.prob_h_given_v(input_X, _w, _hb))
        visibleGen = self.sample_prob(self.prob_v_given_h(hiddenGen, _w, _vb))
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            return sess.run(out), sess.run(visibleGen), sess.run(hiddenGen)
    # function to show the feature detectors of the hidden layer    
    def show_features(self, shape, suptitle, count=-1):
        maxw = np.amax(self.w.T)
        minw = np.amin(self.w.T)
        count = self._output_size if count == -1 or count > self._output_size else count
        ncols = count if count < 14 else 14 
        nrows = count//ncols
        nrows = nrows if nrows > 2 else 3
        fig = plt.figure(figsize=(ncols, nrows), dpi=100)
        grid = Grid(fig, rect=111, nrows_ncols=(nrows, ncols), axes_pad=0.01)

        for i, ax in enumerate(grid):
            x = self.w.T[i] if i<self._input_size else np.zeros(shape)
            x = (x.reshape(1, -1) - minw)/maxw
            ax.imshow(x.reshape(*shape), cmap=mpl.cm.Greys)
            ax.set_axis_off()

        fig.text(0.5,1, suptitle, fontsize=20, horizontalalignment='center')
        fig.tight_layout()
        plt.show()
        return

In [ ]:
inputX = np.array(y)
inputX = inputX.astype(np.float32)

# Create list to hold our RBMs
rbm_list = []

# Define the parameters of the RBMs we will train
rbm_list.append(RBM(3952,3952,0.01,100,32))
rbm_list.append(RBM(3952,3952,0.01,100,32))
rbm_list.append(RBM(3952,3952,0.01,100,32))
#rbm_list.append(RBM(1415,1415,0.1,100,64))
#rbm_list.append(RBM(1415,1415,0.1,100,64))

In [ ]:
outputList = []
error_list = []
#For each RBM in our list
for i in range(0,len(rbm_list)):
    print('RBM', i+1)
    #Train a new one
    rbm = rbm_list[i]
    err = rbm.train(inputX)
    error_list.append(err)
    #Return the output layer
    outputX, reconstructedX, hiddenX = rbm.rbm_output(inputX)
    outputList.append(outputX)
    inputX = hiddenX

RBM 1
Instructions for updating:
Use `tf.cast` instead.
Epoch: 0 reconstruction error: 0.485307
Epoch: 1 reconstruction error: 0.489306
Epoch: 2 reconstruction error: 0.485476
Epoch: 3 reconstruction error: 0.485913
Epoch: 4 reconstruction error: 0.487731
Epoch: 5 reconstruction error: 0.486975
Epoch: 6 reconstruction error: 0.487665
Epoch: 7 reconstruction error: 0.490249
Epoch: 8 reconstruction error: 0.489695
Epoch: 9 reconstruction error: 0.492497
Epoch: 10 reconstruction error: 0.489653
Epoch: 11 reconstruction error: 0.493187
Epoch: 12 reconstruction error: 0.487936
Epoch: 13 reconstruction error: 0.490036
Epoch: 14 reconstruction error: 0.490129
Epoch: 15 reconstruction error: 0.495371
Epoch: 16 reconstruction error: 0.487102
Epoch: 17 reconstruction error: 0.494653
Epoch: 18 reconstruction error: 0.490321
Epoch: 19 reconstruction error: 0.492554
Epoch: 20 reconstruction error: 0.492082
Epoch: 21 reconstruction error: 0.491636
Epoch: 22 reconstruction error: 0.489962
Epoch: 23 r

In [ ]:
outputList

[array([[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]], dtype=float32),
 array([[9.9999654e-01, 9.9999750e-01, 9.9999529e-01, ..., 9.9999774e-01,
         4.7087669e-06, 9.9999499e-01],
        [9.9999654e-01, 9.9999750e-01, 9.9999529e-01, ..., 9.9999774e-01,
         4.7087669e-06, 9.9999499e-01],
        [9.9999654e-01, 9.9999750e-01, 9.9999529e-01, ..., 9.9999774e-01,
         4.7087669e-06, 9.9999499e-01],
        ...,
        [9.9999654e-01, 9.9999750e-01, 9.9999529e-01, ..., 9.9999774e-01,
         4.7087669e-06, 9.9999499e-01],
        [9.9999654e-01, 9.9999750e-01, 9.9999529e-01, ..., 9.9999774e-01,
         4.7087669e-06, 9.9999499e-01],
        [9.9999654e-01, 9.9999750e-01, 9.9999529e-01, ..., 9.9999774e-01,
         4.7087669e-06, 9.9999499e-01]], dtype=float32),
 array([[0.9999988 , 0